In [1]:
import numpy as np
from scipy.optimize import curve_fit

In [2]:
def vector_dot(X,Y):
    FLOP_count = 0
    result=0
    size=len(X)
    if abs(size-len(Y))<0.1:
        #multiplication is possible
        for i in range(size):
            result = X[i]*Y[i] + result
            FLOP_count = FLOP_count+2
    else:
        print('multiplication not possible')
        return
    return (result,{'size':size,'FLOP_count':FLOP_count})

In [3]:
X =np.array([1,2,3,4])
Y=np.array([5,6,7,8])

In [4]:
Y[0]

5

In [5]:
vector_dot(X,Y)[1]["FLOP_count"]

8

In [6]:
def SAXPY(a,X,Y):
    sizeX=len(X)
    sizeY=len(Y)
    FLOP_count=0
    if abs(sizeX-sizeY)<0.1:
        #addition is possible
        for i in range(sizeX):
            Y[i]=Y[i]+a*X[i]
            FLOP_count = FLOP_count+2
    else:
        print('operation not possible')
        return
    return (Y,{'size':sizeX,'FLOP_count':FLOP_count})


In [7]:
vector_dot(X,Y)[0]

70

In [8]:
A=np.identity(4)
A[1][0]=3

In [9]:
A[0]
A

array([[1., 0., 0., 0.],
       [3., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [10]:
def GAXPY(A,X,Y):
    #A is a matrix
    """
    Multiplies A with X and adds it to Y

    Args:
        first_row (array_like): 1-D array representing the first row (or column).

    Returns:
        ndarray: returns a matrix for the operation Y=Y+AX and FLOP count
    """
    sizeX=len(X)
    sizeY=len(Y)
    shapeA=A.shape
    Flop_count=0
    if (shapeA[0]-sizeX)<0.1:
        #multiplication is possible
        mul_array=np.zeros(sizeX)
        for i in range(sizeX):
            result = vector_dot(A[i],X)
            mul_array[i]=result[0]
            Flop_count = Flop_count + result[1]['FLOP_count']
            
        for i in range(sizeX):
            Y[i]=Y[i] + mul_array[i]
            Flop_count = Flop_count + 1
    else:
        print('Operation not possible. Size difference')
        return
    return (Y,{'FLOP_COUNT':Flop_count})
            
        
        
    

In [11]:
GAXPY(A,X,Y) #2X4X4 + 4

(array([ 6, 11, 10, 12]), {'FLOP_COUNT': 36})

In [12]:
mul_array=np.array(4)

In [13]:
def mod_GAXPY(A,X,Y):
    shapeX=X.shape
    shapeY=Y.shape
    shapeA=A.shape
    Flop_count=0
    
    if (shapeX==shapeY)and(shapeA==(shapeX[0],shapeY[0])):
        # shapes are correct, multiplication, addition is possible
        result = np.zeros((shapeX[0],shapeY[0]))
        for i in range(shapeX[0]):
            for j in range(shapeY[0]):
                result[i][j] = X[i]*Y[j]
                Flop_count = Flop_count + 1
        
        #adding result mat with A
        for i in range(shapeX[0]):
            for j in range(shapeY[0]):
                result[i][j] = result[i][j] + A[i][j]
                Flop_count = Flop_count + 1
        
    else:
        print("Matrix shapes and sizes are not compatible")
        return
    
    return (result,{"FLOP_COUNT":Flop_count}) #returns A+X(Y.T)
    

In [14]:
def mat_addition(X,Y):
    if X.shape==Y.shape:
        result=np.zeros(X.shape)
        flop_count=0
        #addition is possible
        for i in range(X.shape[0]):
            for j in range(Y.shape[1]):
                result[i][j] = X[i][j]+Y[i][j]
                flop_count+=1
    else:
        print("shape different")
        return
    return (result,flop_count)

In [15]:
def mod_GAXPY2(A,B,C):
    #returns A+BC
    shapeA=A.shape
    shapeB=B.shape
    shapeC=C.shape
    Flop_count=0
    
    if(shapeB[1]==shapeC[0]) and (shapeA==(shapeB[0],shapeC[1])):
        #operation is possible, all good with shapes
        result=np.zeros(shapeA)
        for i in range(shapeA[0]):
            for j in range(shapeA[1]):
                dot=vector_dot(B[i],C.T[j])
                result[i][j] = dot[0]
                Flop_count=Flop_count+dot[1]['FLOP_count']
        
        BC=(result,Flop_count)
        
        for i in range(shapeA[0]):
            for j in range(shapeA[1]):
                result[i][j]=result[i][j] + A[i][j]
    else:
        print("Matrix shapes and sizes are not compatible")
        return 
    return (BC,result,{'FLOP_count':Flop_count})

In [16]:
def mod_GAXPY3(A,B,C,D):
    #returns A+BCD
    shapeA=A.shape
    shapeB=B.shape
    shapeC=C.shape
    shapeD=D.shape
    Flop_count=0
    
    #multiplying B and C
    BC=mod_GAXPY2(np.zeros((shapeB[0],shapeC[1])),B,C)[0]
    BC_mat=BC[0]
    flopBC=BC[1]
    
    Flop_count=Flop_count+flopBC
    
    #multplying BC and D
    BCD=mod_GAXPY2(np.zeros((shapeB[0],shapeD[1])),BC_mat,D)[0]
    BCD_mat=BCD[0]
    flopBCD=BCD[1]
    
    Flop_count = Flop_count + flopBCD
    
    
    if shapeA==BCD_mat.shape:
        #addition is possible
        for i in range(shapeA[0]):
            for j in range(shapeA[1]):
                BCD_mat[i][j] = BCD_mat[i][j]+A[i][j]
                Flop_count = Flop_count+1
    else:
        print('shapes are incompatible')
        return 
    return (BCD_mat,{"FLOP_count":Flop_count}) #returns A+BCD

In [17]:
#X=(X.T)Y
#Y=Y+aX
#Y=Y+AX
#A=A+X(Y.T)
#A=A+BC
#A=A+BCD


In [18]:
X.shape==Y.shape

True

In [19]:
np.zeros((X.shape[0],Y.shape[0]))[0][0]

0.0

In [20]:
True and False

False

In [21]:
mod_GAXPY(A,X,Y)

(array([[ 7., 11., 10., 12.],
        [15., 23., 20., 24.],
        [18., 33., 31., 36.],
        [24., 44., 40., 49.]]),
 {'FLOP_COUNT': 32})

In [22]:
(X.shape==Y.shape)and(A.shape==(X.shape,Y.shape))

False

In [23]:
A.shape

(4, 4)

In [24]:
X

array([1, 2, 3, 4])

In [25]:
Y

array([ 6, 11, 10, 12])

In [26]:
mod_GAXPY2(A,A,A)

((array([[2., 0., 0., 0.],
         [9., 2., 0., 0.],
         [0., 0., 2., 0.],
         [0., 0., 0., 2.]]),
  128),
 array([[2., 0., 0., 0.],
        [9., 2., 0., 0.],
        [0., 0., 2., 0.],
        [0., 0., 0., 2.]]),
 {'FLOP_count': 128})

In [32]:
(r,f)=mod_GAXPY2(A,A,A)

Matrix shapes and sizes are not compatible


TypeError: cannot unpack non-iterable NoneType object

In [28]:
mod_GAXPY3(A,A,A,A)

(array([[ 2.,  0.,  0.,  0.],
        [12.,  2.,  0.,  0.],
        [ 0.,  0.,  2.,  0.],
        [ 0.,  0.,  0.,  2.]]),
 {'FLOP_count': 272})

In [29]:
A

array([[1., 0., 0., 0.],
       [3., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [30]:
B=np.zeros((4,5))
C=np.zeros((5,6))
D=np.zeros((6,7))
A=np.zeros((4,7))

In [ ]:
A=np.zeros((4,4))

In [31]:
mod_GAXPY3(A,B,C,D)

(array([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]),
 {'FLOP_count': 604})

In [33]:
#1. write a python function for C=AB using a special algorithm using Strassen multiplication
#2. plot N_Flops of coventional matrix-matrix multiplication C[i][j] = Sum(over k)(A_ik)(B_kj) along with N_Flops of Strassen multiplication


In [ ]:
# u = [1:N/2] v=[N/2:N]
# P1 = (A_(u,u) + A_(v,v))(B_(u,u)+B_(v,v))
# P2 = (A_(v,u) + A_(v,v))B_(u,u)
# P3 = A_(u,u)(B_(u,v)-B_(v,v))
# P4 = A_(v,v)